# Neural Networks with Word2Vec

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from nltk import word_tokenize
from gensim.models import word2vec

In [2]:
df_tri = pd.read_csv('branded_tweets_clean.csv')
df_bin = pd.read_csv('binary_tweets.csv')

df_tri.drop(columns=['hashtag_count', 'mention_count', 'weblink_count', 
                     'tweet_length', 'punct_count'], inplace=True)
df_bin.drop(columns=['hashtag_count', 'mention_count', 'weblink_count', 
                     'tweet_length', 'punct_count'], inplace=True)

def clean_string(text):
    clean_text = text.replace("[",'').replace("]",'').replace("'",'').replace(",",'')
    return clean_text

df_bin['tweet_tokens'] = df_bin['tweet_tokens'].map(clean_string)
df_tri['tweet_tokens'] = df_tri['tweet_tokens'].map(clean_string)

In [3]:
df_tri.head()

tweet       brand_product  \
0  .@wesley83 I have a 3G iPhone. After 3 hrs twe...              iPhone   
1  @jessedee Know about @fludapp ? Awesome iPad/i...  iPad or iPhone App   
2  @swonderlin Can not wait for #iPad 2 also. The...                iPad   
3  @sxsw I hope this year's festival isn't as cra...  iPad or iPhone App   
4  @sxtxstate great stuff on Fri #SXSW: Marissa M...              Google   

   sentiment                                       tweet_tokens  
0          0  wesley83 3g iphone 3 hr tweeting riseaustin de...  
1          1  jessedee know fludapp awesome ipadiphone app y...  
2          1              swonderlin wait ipad 2 also sale sxsw  
3          0  sxsw hope year festival isnt crashy year iphon...  
4          1  sxtxstate great stuff fri sxsw marissa mayer g...

In [4]:
df_bin.head()

tweet       brand_product  \
0  .@wesley83 I have a 3G iPhone. After 3 hrs twe...              iPhone   
1  @jessedee Know about @fludapp ? Awesome iPad/i...  iPad or iPhone App   
2  @swonderlin Can not wait for #iPad 2 also. The...                iPad   
3  @sxsw I hope this year's festival isn't as cra...  iPad or iPhone App   
4  @sxtxstate great stuff on Fri #SXSW: Marissa M...              Google   

   sentiment                                       tweet_tokens  
0          0  wesley83 3g iphone 3 hr tweeting riseaustin de...  
1          1  jessedee know fludapp awesome ipadiphone app y...  
2          1              swonderlin wait ipad 2 also sale sxsw  
3          0  sxsw hope year festival isnt crashy year iphon...  
4          1  sxtxstate great stuff fri sxsw marissa mayer g...

In [5]:
bag_of_words = df_bin['tweet_tokens']
words_list = []
for string_ele in bag_of_words:
    split_up_words = string_ele.split()
    for i in range(0, len(split_up_words)):
        words_list.append(split_up_words[i])
        
total_vocabulary = set(words_list)

In [6]:
len(words_list)

101802

In [7]:
len(total_vocabulary)

8929

In [39]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras import regularizers
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

In [9]:
y_bin = df_bin['sentiment']
y_tri = pd.get_dummies(df_tri['sentiment']).values

In [51]:
tokenizer_bin = text.Tokenizer(num_words=6500)
tokenizer_bin.fit_on_texts(list(df_bin['tweet_tokens']))
list_tokenized_tweets_bin = tokenizer_bin.texts_to_sequences(df_bin['tweet_tokens'])
X_bin = sequence.pad_sequences(list_tokenized_tweets_bin)

In [52]:
model_bin = Sequential()

In [53]:
embedding_size = 100
model_bin.add(Embedding(6500, embedding_size))
model_bin.add(LSTM(30))
model_bin.add(Dropout(0.5))
model_bin.add(Dense(20, activation='relu'))
model_bin.add(Dropout(0.5))
model_bin.add(Dense(1, activation='sigmoid'))

In [54]:
model_bin.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [55]:
model_bin.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 100)         650000    
_________________________________________________________________
lstm_13 (LSTM)               (None, 30)                15720     
_________________________________________________________________
dropout_28 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 20)                620       
_________________________________________________________________
dropout_29 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 21        
Total params: 666,361
Trainable params: 666,361
Non-trainable params: 0
_________________________________________________________________


In [56]:
model_bin.fit(X_bin, y_bin, epochs=40, batch_size=256, validation_split=0.3)

Train on 5831 samples, validate on 2499 samples
Epoch 1/40
5831/5831 [==============================] - 3s 518us/step - loss: 0.6842 - acc: 0.6042 - val_loss: 0.6619 - val_acc: 0.6495
Epoch 2/40
5831/5831 [==============================] - 1s 253us/step - loss: 0.6547 - acc: 0.6359 - val_loss: 0.6359 - val_acc: 0.6515
Epoch 3/40
5831/5831 [==============================] - 2s 274us/step - loss: 0.6216 - acc: 0.6438 - val_loss: 0.6031 - val_acc: 0.6623
Epoch 4/40
5831/5831 [==============================] - 2s 272us/step - loss: 0.5290 - acc: 0.7477 - val_loss: 0.5898 - val_acc: 0.6931
Epoch 5/40
5831/5831 [==============================] - 2s 270us/step - loss: 0.4420 - acc: 0.8294 - val_loss: 0.6024 - val_acc: 0.7011
Epoch 6/40
5831/5831 [==============================] - 2s 262us/step - loss: 0.3582 - acc: 0.8643 - val_loss: 0.6856 - val_acc: 0.6995
Epoch 7/40
5831/5831 [==============================] - 2s 302us/step - loss: 0.2965 - acc: 0.8870 - val_loss: 0.7871 - val_acc: 0.7023


In [57]:
model_bin2 = Sequential()
model_bin2.add(Embedding(6500, 32))
model_bin2.add(LSTM(15))
model_bin2.add(Dropout(0.5))
model_bin2.add(Dense(10, activation='relu'))
model_bin2.add(Dropout(0.5))
model_bin2.add(Dense(1, activation='sigmoid'))

In [58]:
model_bin2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [59]:
model_bin2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 32)          208000    
_________________________________________________________________
lstm_14 (LSTM)               (None, 15)                2880      
_________________________________________________________________
dropout_30 (Dropout)         (None, 15)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                160       
_________________________________________________________________
dropout_31 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 211,051
Trainable params: 211,051
Non-trainable params: 0
_________________________________________________________________


In [60]:
model_bin2.fit(X_bin, y_bin, epochs=30, batch_size=256, validation_split=0.3)

Train on 5831 samples, validate on 2499 samples
Epoch 1/30
5831/5831 [==============================] - 3s 482us/step - loss: 0.6866 - acc: 0.6109 - val_loss: 0.6766 - val_acc: 0.6495
Epoch 2/30
5831/5831 [==============================] - 1s 148us/step - loss: 0.6690 - acc: 0.6357 - val_loss: 0.6504 - val_acc: 0.6495
Epoch 3/30
5831/5831 [==============================] - 1s 161us/step - loss: 0.6567 - acc: 0.6321 - val_loss: 0.6430 - val_acc: 0.6495
Epoch 4/30
5831/5831 [==============================] - 1s 168us/step - loss: 0.6366 - acc: 0.6381 - val_loss: 0.6246 - val_acc: 0.6515
Epoch 5/30
5831/5831 [==============================] - 1s 166us/step - loss: 0.5994 - acc: 0.6534 - val_loss: 0.5949 - val_acc: 0.6731
Epoch 6/30
5831/5831 [==============================] - 1s 163us/step - loss: 0.5315 - acc: 0.7366 - val_loss: 0.5848 - val_acc: 0.7035
Epoch 7/30
5831/5831 [==============================] - 1s 169us/step - loss: 0.4612 - acc: 0.7973 - val_loss: 0.6011 - val_acc: 0.6879


In [64]:
model_bin3 = Sequential()
model_bin3.add(Embedding(6500, 16))
model_bin3.add(LSTM(10))
model_bin3.add(Dropout(0.5))
model_bin3.add(Dense(5, activation='relu'))
model_bin3.add(Dropout(0.5))
model_bin3.add(Dense(1, activation='sigmoid'))

In [65]:
model_bin3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bin3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, None, 16)          104000    
_________________________________________________________________
lstm_16 (LSTM)               (None, 10)                1080      
_________________________________________________________________
dropout_34 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 5)                 55        
_________________________________________________________________
dropout_35 (Dropout)         (None, 5)                 0         
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 6         
Total params: 105,141
Trainable params: 105,141
Non-trainable params: 0
_________________________________________________________________


In [66]:
model_bin3.fit(X_bin, y_bin, epochs=30, batch_size=256, validation_split=0.3)

Train on 5831 samples, validate on 2499 samples
Epoch 1/30
5831/5831 [==============================] - 3s 453us/step - loss: 0.6862 - acc: 0.6277 - val_loss: 0.6766 - val_acc: 0.6495
Epoch 2/30
5831/5831 [==============================] - 1s 137us/step - loss: 0.6725 - acc: 0.6349 - val_loss: 0.6602 - val_acc: 0.6495
Epoch 3/30
5831/5831 [==============================] - 1s 130us/step - loss: 0.6617 - acc: 0.6333 - val_loss: 0.6479 - val_acc: 0.6495
Epoch 4/30
5831/5831 [==============================] - 1s 146us/step - loss: 0.6585 - acc: 0.6356 - val_loss: 0.6449 - val_acc: 0.6495
Epoch 5/30
5831/5831 [==============================] - 1s 137us/step - loss: 0.6428 - acc: 0.6364 - val_loss: 0.6291 - val_acc: 0.6495
Epoch 6/30
5831/5831 [==============================] - 1s 139us/step - loss: 0.6211 - acc: 0.6361 - val_loss: 0.6106 - val_acc: 0.6495
Epoch 7/30
5831/5831 [==============================] - 1s 144us/step - loss: 0.5842 - acc: 0.6369 - val_loss: 0.5963 - val_acc: 0.6495


In [85]:
model_bin4 = Sequential()
model_bin4.add(Embedding(6500, 16))
model_bin4.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l1(0.005)))
model_bin4.add(Dropout(0.5))
model_bin4.add(LSTM(5))
model_bin4.add(Dropout(0.3))
model_bin4.add(Dense(4, activation='relu', kernel_regularizer=regularizers.l1(0.005)))
model_bin4.add(Dropout(0.3))
model_bin4.add(Dense(1, activation='sigmoid'))

In [86]:
model_bin4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bin4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, None, 16)          104000    
_________________________________________________________________
dense_53 (Dense)             (None, None, 8)           136       
_________________________________________________________________
dropout_53 (Dropout)         (None, None, 8)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 5)                 280       
_________________________________________________________________
dropout_54 (Dropout)         (None, 5)                 0         
_________________________________________________________________
dense_54 (Dense)             (None, 4)                 24        
_________________________________________________________________
dropout_55 (Dropout)         (None, 4)                 0         
__________

In [92]:
model_bin4.fit(X_bin, y_bin, epochs=25, batch_size=150, validation_split=0.25)

Train on 6247 samples, validate on 2083 samples
Epoch 1/25
6247/6247 [==============================] - 3s 501us/step - loss: 0.4240 - acc: 0.7898 - val_loss: 0.7745 - val_acc: 0.6793
Epoch 2/25
6247/6247 [==============================] - 3s 452us/step - loss: 0.4300 - acc: 0.7879 - val_loss: 0.7760 - val_acc: 0.7033
Epoch 3/25
6247/6247 [==============================] - 3s 484us/step - loss: 0.4281 - acc: 0.7903 - val_loss: 0.7734 - val_acc: 0.6880
Epoch 4/25
6247/6247 [==============================] - 3s 493us/step - loss: 0.4246 - acc: 0.7937 - val_loss: 0.7804 - val_acc: 0.6899
Epoch 5/25
6247/6247 [==============================] - 3s 511us/step - loss: 0.4210 - acc: 0.7972 - val_loss: 0.8084 - val_acc: 0.7072
Epoch 6/25
6247/6247 [==============================] - 3s 486us/step - loss: 0.4262 - acc: 0.8031 - val_loss: 0.7726 - val_acc: 0.6855
Epoch 7/25
6247/6247 [==============================] - 3s 560us/step - loss: 0.4194 - acc: 0.8119 - val_loss: 0.7964 - val_acc: 0.7024
